In [1]:
import os
import utils.cv_reviewer_utils as cv_utils
from IPython.display import Markdown, display
import json
import warnings

In [2]:

# Disable SSL warnings (optional but not recommended in production)
warnings.filterwarnings("ignore")

In [3]:
#if its published somewhere
#job_description = get_job_description("url")

job_description = """

Job Title: Junior Data Engineer
Location: Remote
Job Type: Full-time
Team: Data Engineering
Reports to: Senior Data Engineer

About the Role:

We are looking for a Junior Data Engineer to join our growing data team. You will support the design, development, and maintenance of data pipelines and infrastructure that power data analytics, reporting, and data-driven decision-making across the company.
This is a great opportunity for someone early in their career to learn and grow while working on real-world data challenges in a collaborative and agile environment.

Key Responsibilities
- Assist in building, maintaining, and optimizing ETL/ELT pipelines.
- Work closely with data analysts, scientists, and engineers to support data ingestion and transformation workflows.
- Help ensure the reliability and quality of data across systems.
- Monitor and troubleshoot data pipelines and processes.
- Contribute to documentation of data flows, models, and architecture.
- Learn and apply best practices in data engineering, including security and scalability.

Requirements:

- Basic knowledge of SQL and at least one programming language (Python preferred).
- Familiarity with data storage systems (e.g., relational databases, cloud storage).
- Understanding of data processing concepts and tools.
- Willingness to learn and grow in a fast-paced environment.
- Good communication and problem-solving skills.
- Bachelor’s degree in Computer Science, Engineering, Mathematics, or related field (or equivalent practical experience).

Nice to Have:

- Exposure to cloud platforms like AWS, GCP, or Azure.
- Experience with version control (e.g., Git).
- Basic knowledge of data modeling concepts.
- Familiarity with tools like Airflow, dbt, or Spark.

"""

In [4]:

# in lower case
mandatory_keywords = ["spark","python"]
language = "spanish"

landing_path = './landing'
execution_mode = "online" # online / offline
        
# "llama3.2" or gpt-4o", "gpt-4-turbo", "gpt-4o-mini"...
model = "gpt-4-turbo"

In [5]:
analysis_text = cv_utils.analyze_candidates(model, job_description, mandatory_keywords, landing_path, language,execution_mode)
display(Markdown(analysis_text))

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4-turbo via OpenAI API...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4-turbo via OpenAI API...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4-turbo via OpenAI API...


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4-turbo via OpenAI API...


### 📊 Candidate Evaluations


**🧑‍💼 Candidate #1: Thaddeus Drake.pdf – 85%**

El candidato tiene una experiencia relevante como Ingeniero de Datos Junior, trabajando con tecnologías y herramientas que coinciden en gran medida con la descripción del trabajo. Ha demostrado habilidades en SQL, Python, AWS, y herramientas de ETL como Apache NiFi y Airflow, que son cruciales para el rol. Aunque no se menciona Spark, su experiencia con tecnologías similares sugiere un potencial de adaptación rápida.

**📝 Recommended Questions**
- ¿Puede describir un desafío que enfrentó mientras trabajaba con ETL y cómo lo superó?
- ¿Cómo asegura la calidad y fiabilidad de los datos en los sistemas que administra?

---

**🧑‍💼 Candidate #2: Elara Quinn.pdf – 70%**

El candidato tiene experiencia relevante en ingeniería de datos, especialmente con herramientas como Apache Hadoop, Apache NiFi, MySQL y AWS, que son pertinentes para el rol ofrecido. Sin embargo, hay una falta de experiencia explícita con Python y herramientas específicas como Airflow o Spark que son deseables para el puesto. La experiencia con AWS es un punto a favor, aunque no se menciona el uso de servicios específicos más allá de AWS Lambda.

**📝 Recommended Questions**
- ¿Puede describir un proyecto donde haya utilizado Python, si lo ha hecho?
- ¿Tiene experiencia con herramientas de orquestación de datos como Airflow o Spark?

---

**🧑‍💼 Candidate #3: john smith.pdf – 65%**

El candidato tiene más de 5 años de experiencia como Ingeniero de Datos, superando el nivel de junior requerido para el puesto. Ha trabajado con SQL, Java, Apache Spark, Hadoop y Python, lo que coincide con varias de las tecnologías mencionadas en la descripción del trabajo. Sin embargo, no se menciona experiencia específica con plataformas en la nube ni con herramientas como Airflow o dbt, que son deseables para el puesto.

**📝 Recommended Questions**
- ¿Puede describir un proyecto donde haya utilizado ETL/ELT pipelines?
- ¿Tiene experiencia trabajando en entornos de nube como AWS, GCP o Azure?
- ¿Cómo asegura la calidad y fiabilidad de los datos en los sistemas que administra?

---

**🧑‍💼 Candidate #4: alan susa.pdf – 55%**

El candidato tiene experiencia avanzada como Ingeniero de Datos, superando el nivel de entrada requerido para el puesto de Ingeniero de Datos Junior. Ha trabajado con tecnologías relevantes como Spark, Python, y plataformas en la nube como AWS, que son compatibles con las mencionadas en la descripción del trabajo. Sin embargo, su experiencia y habilidades pueden estar por encima del nivel junior, lo que podría no ser ideal para un rol que busca a alguien al principio de su carrera.

**📝 Recommended Questions**
- ¿Puede describir un proyecto donde haya tenido que aprender una nueva tecnología o herramienta desde cero?
- ¿Cómo maneja la transición de trabajar en proyectos de gran escala a proyectos más pequeños o roles con menos responsabilidad?
- ¿Cuál es su experiencia trabajando en entornos colaborativos y ágiles?

---
